# Data preparation

This dataset needed to be cleaned, and additional work needed to be performed in order to aggregate the two datasets.  This notebook is included for reference, but the other notebook should work without it.  


In [1]:
# Import all the things

%matplotlib inline
import math

from datetime import datetime
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as stats
from sqlalchemy import create_engine
plt.style.use('seaborn-darkgrid')
import pyproj
import turfpy
from turfpy.measurement import boolean_point_in_polygon
from geojson import Point, Polygon, MultiPolygon, Feature, FeatureCollection
from statsmodels.graphics.gofplots import qqplot_2samples, qqplot

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_regression,  mutual_info_regression, chi2
from sklearn.model_selection import train_test_split

from pyproj import Proj
from pyproj import CRS, Transformer

# For displaying LaTeX when outputting 
from IPython.display import display, Markdown


In [2]:
# Collected data were added into a normalised SQL database. For analysis they need to be denormalised

query = """
    SELECT 
        a.id, address, sale_date, asked_price, sold_price, rooms, living_space, has_balcony, 
        floor, avgift, driftskostnad, built, total_floors, has_lift, lat, lng
    FROM sales s
    INNER JOIN apartments a on s.apartment_id = a.id
    INNER JOIN buildings b on a.building_id = b.id
"""
engine = create_engine('sqlite:///data/records3.db')
data = pd.read_sql(query, engine)
data

,id,address,sale_date,asked_price,sold_price,rooms,living_space,has_balcony,floor,avgift,driftskostnad,built,total_floors,has_lift,lat,lng
0,1,Splintvedsgatan 6,2021-07-29,2795000.0,2900000,2.0,52.0,1.0,2.0,3532.0,6660.0,2018-01-01,3.0,1,57.624273,12.009470
1,2,Ponnygatan 2E,2021-07-29,2390000.0,2450000,3.0,78.0,1.0,3.0,4829.0,NaN,1966-01-01,3.0,0,57.646285,12.012905
2,3,Ålegårdsgatan 239,2021-07-28,1995000.0,2550000,2.0,65.0,NaN,-1.0,4308.0,15648.0,2100-01-01,2.0,0,57.624449,12.022623
3,4,Kvarnbygatan 39,2021-07-27,NaN,2350000,1.0,34.0,NaN,4.0,2561.0,1440.0,2017-01-01,NaN,1,57.658056,12.029191
4,5,Vänortsgatan 8,2021-07-27,1100000.0,1225000,1.0,19.0,NaN,3.0,1873.0,3600.0,1952-01-01,3.0,0,57.663665,12.012079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5007,5008,Arvesgärde 30,2021-09-03,2295000.0,2295000,3.0,81.0,1.0,2.0,5378.0,6900.0,1972-01-01,NaN,1,57.751690,11.919390
5008,5009,Distansgatan 23 D,2021-09-15,2095000.0,2050000,1.0,41.0,NaN,1.0,2606.0,5160.0,2016-01-01,NaN,1,57.664020,11.934450
5009,5010,Smörslottsgatan 53,2021-10-07,3595000.0,4325000,3.0,79.0,1.0,4.0,5448.0,4200.0,2004-01-01,4.0,1,57.715014,12.056231
5010,5011,Fullmånegatan 18,2021-11-24,2500000.0,2700000,3.0,77.0,1.0,NaN,4837.0,3000.0,1994-01-01,3.0,1,57.745279,12.024910


In [3]:
# Avgift is given as a monthly cost, when for comparison 
# we need it to be cost per sqm to be comparable

avgift_sqm = data['avgift'] / data['living_space']
data['avgift_sqm'] = avgift_sqm


In [4]:
# sqlite stores dates as strings, so casting to a date seems necessary. 
# Then we can filter to only include properties from September - December 2021
# This is necessary, because some data from earlier iterations of the project remains in the database

data['sale_date'] = pd.to_datetime(data['sale_date'])
data_2021 = data[data['sale_date'].dt.year == 2021]
data_2021 = data[data['sale_date'].dt.month > 8]

data_2021.count()

id               3104
address          3104
sale_date        3104
asked_price      2865
sold_price       3104
rooms            3102
living_space     3103
has_balcony      1857
floor            2558
avgift           3103
driftskostnad    2583
built            3104
total_floors     2278
has_lift         3104
lat              3104
lng              3104
avgift_sqm       3102
dtype: int64

### Voting district data

The data was extracted using the turfpy library, although there did appear to be some compatibility issues with the GeoJSON types so some data preparation was required for this.

In addition, government data in Sweden is stored using the SWEREF coordinates, whereas the data from Hemnet had latitude and longitude.  The Python "turfpy" library was used to  handle the conversion.

In order to improve performance, it was assumed that only Västra Götalands Län would be a useful, so the data for all other counties was excluded.  Each property was assigned a valdistrikt

In [5]:
# See which voting district they are in, and add that to the dataset
def get_voting_districts():
    with open('data/alla_valdistrikt.json', encoding='utf-8') as f:
        return json.load(f)


In [6]:
# Convert the Valdistrikt data from JSON to GeoJSON objects
def get_features(voting_districts):
    polygons = []
    for feature in voting_districts['features']:
        try:
            feature_type = MultiPolygon if feature['geometry']['type'] == 'MultiPolygon' else Polygon
            # We don't need almost all of the regions, all the flats will be in Västra Götaland
            if feature['properties']['LAN_NAMN'] != 'Västra Götalands län':
                continue
            f = Feature(
                geometry=feature_type(feature['geometry']['coordinates']),
                properties=feature['properties']
            )
            polygons.append(f)
        except Exception as e:
            print(e)
            print(feature)
            break
    return polygons


In [7]:
# It seems that the Swedish voting data are not stored in lat/lng, but rather SWEREF99
# So we'll need to convert it

def convert_to_sweref99(latlng):
    lat, lng = latlng
    rt90 = CRS.from_epsg(3021)
    wgs84 = CRS.from_epsg(4326)
    sweref = CRS.from_epsg(3006)
    transformer = Transformer.from_crs(wgs84, sweref)
    return transformer.transform(lat, lng)

convert_to_sweref99((57.62427303, 12.009470))

(6390818.000015095, 321418.99712134723)

In [8]:
def find_valdistrikts(df, polygons):
    valdistrikts = []
    for i, indexrow in enumerate(df.iterrows()):
        index, row = indexrow
        lat = row['lat']
        lng = row['lng']
        p, q = convert_to_sweref99((lat, lng))
        if i % 100 == 0:
            print(f'{i} Looking for {row["address"]}')
        for i, feature in enumerate(polygons):
            try:
                if boolean_point_in_polygon(Point((q,p)),feature):
                    valdistrikt = feature.properties["VD_NAMN"]
                    valdistrikts.append(valdistrikt)
            except Exception as e:
                print(e)
                break
    df['valdistrikt'] = valdistrikts
    return df


In [9]:
polygons = get_features(get_voting_districts())

start = datetime.now()
data_2021 = find_valdistrikts(data_2021, polygons)
finished = datetime.now()

print(finished - start)
data_2021.head()

0 Looking for Eklanda Hage 145C
100 Looking for Pinnharvsgatan 3A
200 Looking for Kvarnbyvallen 5
300 Looking for Bergskroken 6
400 Looking for Streteredsvägen 1C
500 Looking for Djäknegatan 1 A
600 Looking for Broslättsgatan 17
700 Looking for Lammevallsgatan 42 F
800 Looking for Höjdgatan 47
900 Looking for Sjömansgatan 3
1000 Looking for Bandtraktorgatan 1
1100 Looking for Storhöjdsgatan 7A
1200 Looking for Seglaregatan 20
1300 Looking for Sergelsgatan 1B
1400 Looking for Föreningsgatan 16E
1500 Looking for Basungatan 5
1600 Looking for Sixten Camps Gata 12
1700 Looking for Björnekärrsgatan 6C
1800 Looking for Generalsgatan 4
1900 Looking for Västgötagatan 9
2000 Looking for Kvadrantgatan 24
2100 Looking for Solventilsgatan 16
2200 Looking for Klareborgsgatan 34 C
2300 Looking for Kolonigatan 9E
2400 Looking for Soldathemsgatan 20
2500 Looking for Träringen 22
2600 Looking for Litteraturgatan 220
2700 Looking for Jöns Rundbäcks Plats 1
2800 Looking for Hisingsgatan 23
2900 Looking f

/tmp/ipykernel_149/1004532934.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['valdistrikt'] = valdistrikts


,id,address,sale_date,asked_price,sold_price,rooms,living_space,has_balcony,floor,avgift,driftskostnad,built,total_floors,has_lift,lat,lng,avgift_sqm,valdistrikt
562,563,Eklanda Hage 145C,2020-12-30,4350000.0,4510000,5.0,116.0,NaN,NaN,6296.0,15456.0,1997-01-01,NaN,0,57.654146,11.963737,54.275862,Eklanda Västra
563,564,Högenvägen 23,2020-12-30,2595000.0,2800000,3.0,72.0,1.0,2.0,3735.0,7140.0,2100-01-01,2.0,0,57.607780,12.058046,51.875000,Brattås Södra
564,565,Frälsegårdsgatan 1 B,2020-12-29,1150000.0,2300000,2.0,54.0,NaN,2.0,2539.0,4980.0,1957-01-01,3.0,0,57.660761,12.001016,47.018519,Bosgården Södra
565,566,Toltorpsgatan 50 B,2020-12-29,900000.0,2000000,1.0,34.0,NaN,NaN,1781.0,NaN,2100-01-01,NaN,0,57.673123,11.978522,52.382353,Toltorp Norra
566,567,Broslättsgatan 36 lgh 1203,2020-12-23,3150000.0,3150000,2.0,54.0,1.0,2.0,3506.0,NaN,2100-01-01,2.0,1,57.654087,12.011442,64.925926,Broslätt Västra


In [10]:
# csv_data = data_2021.to_csv(index=False)
# with open('data/data.csv', 'w', encoding='utf-8') as f:
#     f.write(csv_data)

In [11]:
data = pd.read_csv('data/data.csv')
data.head()

,id,address,sale_date,asked_price,sold_price,rooms,living_space,has_balcony,floor,avgift,driftskostnad,built,total_floors,has_lift,lat,lng,avgift_sqm,valdistrikt
0,563,Eklanda Hage 145C,2020-12-30,4350000.0,4510000,5.0,116.0,NaN,NaN,6296.0,15456.0,1997-01-01,NaN,0,57.654146,11.963737,54.275862,Eklanda Västra
1,564,Högenvägen 23,2020-12-30,2595000.0,2800000,3.0,72.0,1.0,2.0,3735.0,7140.0,2100-01-01,2.0,0,57.607780,12.058046,51.875000,Brattås Södra
2,565,Frälsegårdsgatan 1 B,2020-12-29,1150000.0,2300000,2.0,54.0,NaN,2.0,2539.0,4980.0,1957-01-01,3.0,0,57.660761,12.001016,47.018519,Bosgården Södra
3,566,Toltorpsgatan 50 B,2020-12-29,900000.0,2000000,1.0,34.0,NaN,NaN,1781.0,NaN,2100-01-01,NaN,0,57.673123,11.978522,52.382353,Toltorp Norra
4,567,Broslättsgatan 36 lgh 1203,2020-12-23,3150000.0,3150000,2.0,54.0,1.0,2.0,3506.0,NaN,2100-01-01,2.0,1,57.654087,12.011442,64.925926,Broslätt Västra
